In [18]:
import pandas as pd
import re
import jieba

def data_process(file='附件4.xlsx'):
    data = pd.read_excel(file,  index_col=0, sep = ',', encoding = 'GB18030')
    

    #data_dup = data['答复意见'].drop_duplicates()
    #去数字字母
    data_qu123a = data['答复意见'].apply(lambda x: re.sub('[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+', "", x))
    #jieba.load_userdict('newdic1.txt')
    data_cut = data_qu123a.apply(lambda x: jieba.lcut(x))

    #设置不存在于停用词表的分隔符，避免 ，被默认为分隔符而报错
    stopWords = pd.read_csv('stopword.txt', encoding='GB18030', sep='hahaha', header=None)
    #列表使用+拼接
    stopWords = [' ', '\n', '\t', '\r\n', '\u3000', '＂', '–','现将','在','的','您好','你好','您','网友','留言','已收悉','：',
                '反映','的问题','平台','同志','现','领导'] + list(stopWords.iloc[:, 0])
    
    #分词
    data_after_stop = data_cut.apply(lambda x: [i for i in x if i not in stopWords])
    
    #用空格分割列表中的词语
    #adata = data_after_stop.apply(lambda x: ' '.join(x))
    return data_after_stop

In [19]:
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel

data_after_stop = data_process()
data_after_stop.to_csv('分词后.csv')



D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [20]:
data_after_stop

留言编号
2549      [问政, 西地省, 栏目, 胡华衡, 书记, 区景蓉, 花苑, 物业管理, 调查核实, 情况...
2554      [区潇楚, 南路, 洋湖, 段, 修好, 区洋湖, 街道, 高度重视, 组织, 精干, 力量...
2555      [市民, 请, 加快, 提高, 民营, 幼儿园, 教师, 待遇, 来信, 收悉, 回复, 改...
2557      [问政, 西地省, 收悉, 市住, 建局, 交由, 市, 房屋交易, 管理中心, 办理, 相...
2574      [收悉, 具体内容, 答复, 来信, 建议, 白竹坡, 路口, 更名, 马坡岭, 小学, 原...
                                ...                        
181267                            [收悉, 已转, 区委, 区, 人民政府, 调查]
181603                                [收悉, 转市, 交通运输, 局, 调查]
184423    [获悉, 对县, 文盛, 小学, 特色, 班, 质疑, 网帖, 我局, 高度重视, 责成, ...
185799    [西地省, 问政, 西地省, 栏目组, 网民, 贵, 栏目, 咨询, 中央, 转移, 支付,...
185986    [我厅, 高度重视, 相关, 部门, 调查, 研究, 回复, 我厅, 县城, 朱良桥, 公路...
Name: 答复意见, Length: 2816, dtype: object

In [21]:
data_com = pd.read_csv('分词后.csv',  header=None,encoding = 'utf-8')
data_com.columns = ['留言编号','comments']

In [22]:
topic = []
for i in range(0, len(data_after_stop)):
    mid = data_com['comments'][i:i+1].str.split(', ')
    dictionary = Dictionary(mid)                     # 生成词典
    bow = [dictionary.doc2bow(j) for j in mid]    # 将文档转成数值型预料库

    data_model = LdaModel(corpus=bow, id2word=dictionary, num_topics=2)    # 构建LDA主题模型
    topic.append(data_model.print_topic(0)+'\n'+data_model.print_topic(1))    # 打印主题




In [23]:
from pandas.core.frame import DataFrame
data_topic = DataFrame(topic)
data_topic.columns = ['主题']
data_new = pd.concat([data_com, data_topic], axis=1)                          #横向拼接
data_new.to_excel('主题2.xlsx',encoding = 'GB18030')

In [13]:
data_new = pd.read_excel('主题2.xlsx',encoding = 'GB18030')

In [14]:
data_new

,Unnamed: 0,留言编号,comments,主题
0,0,2549,"['问政', '西地省', '栏目', '胡华衡', '书记', '区景蓉', '花苑', ...","0.032*""'业委会'"" + 0.029*""'业主'"" + 0.023*""'业主大会'"" ..."
1,1,2554,"['区潇楚', '南路', '洋湖', '段', '修好', '区洋湖', '街道', '高...","0.054*""'施工'"" + 0.028*""'道路'"" + 0.026*""'项目'"" + 0..."
2,2,2555,"['市民', '请', '加快', '提高', '民营', '幼儿园', '教师', '待遇...","0.043*""'民办'"" + 0.042*""'幼儿园'"" + 0.040*""'教师'"" + ..."
3,3,2557,"['问政', '西地省', '收悉', '市住', '建局', '交由', '市', '房屋...","0.053*""'购房'"" + 0.035*""'市'"" + 0.033*""'首次'"" + 0...."
4,4,2574,"['收悉', '具体内容', '答复', '来信', '建议', '白竹坡', '路口', ...","0.048*""'来信'"" + 0.047*""'马坡岭'"" + 0.045*""'小学'"" + ..."
...,...,...,...,...
2811,2811,181267,"['收悉', '已转', '区委', '区', '人民政府', '调查']","0.174*""'已转'"" + 0.168*""['收悉'"" + 0.167*""'区委'"" + ..."
2812,2812,181603,"['收悉', '转市', '交通运输', '局', '调查']","0.218*""'交通运输'"" + 0.201*""'局'"" + 0.196*""['收悉'"" +..."
2813,2813,184423,"['获悉', '对县', '文盛', '小学', '特色', '班', '质疑', '网帖'...","0.039*""'学生'"" + 0.028*""'社团活动'"" + 0.023*""'社团'"" +..."
2814,2814,185799,"['西地省', '问政', '西地省', '栏目组', '网民', '贵', '栏目', '...","0.070*""'资金'"" + 0.047*""'支付'"" + 0.042*""'燃油税'"" + ..."


In [ ]:
'''
temp = []
for i in range(0, 2750):
    mid = data_com['comments'][i:i+1].str.split(' ')
    temp.append(mid)

temp_split = DataFrame(temp)

print(temp_split)
    #print(mid.dtype)
    #print('-------------------------------')
    #dictionary = Dictionary(mid)

#mid0 = data_com['comments'][0:1].str.split(' ')
print(mid0.dtype)
#dictionary = Dictionary(mid0)
#bow = [dictionary.doc2bow(j) for j in mid0]
#data_model = LdaModel(corpus=bow, id2word=dictionary, num_topics=3)    # 构建LDA主题模型
#data_model.print_topic(1)    # 打印主题
'''

In [15]:
import pandas as pd

left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                       'A': ['A0', 'A1', 'A2', 'A3'],
                       'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']})
left



,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [16]:
right

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


In [17]:
pd.merge(left, right, on='key')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3
